In [1]:
# -*- coding: utf-8 -*-
import codecs
import re
import sys
import time
from collections import Counter, defaultdict, OrderedDict

# Preprocessing corpus

In [2]:
STOPWORDS_FILENAME = 'pap/stopwords1.txt'

stopwords = set([line.strip() for line in codecs.open(STOPWORDS_FILENAME, encoding='utf-8').readlines()])

In [3]:
def remove_special_chars(text):
    return re.compile(r'[\W\d]+', re.UNICODE).sub(' ', text)

In [4]:
def remove_whitespace(text):
    return re.compile(r'[\s\n]+', re.UNICODE).sub(' ', text)

In [5]:
def remove_stopwords(words):
    return [word for word in words if word not in stopwords]

In [6]:
def preprocess_corpus(corpus):
    corpus = corpus.strip().lower()
    corpus = remove_special_chars(corpus)
    corpus = remove_whitespace(corpus)
    words = corpus.split()
    return remove_stopwords(words)

In [7]:
text = u'Jesień w 2017 Japonii jest przepiękna.'
for word in preprocess_corpus(text):
    print word

jesień
japonii
przepiękna


In [8]:
FORMS_FILENAME = 'pap/odm.txt'

In [9]:
class Dictionary(object):
    def __init__(self):
        self.basic_forms = []
        self.forms = defaultdict(lambda: [])

        with codecs.open(FORMS_FILENAME, encoding='utf-8') as forms_file:
            lines = forms_file.readlines()
            for line in lines:
                forms = line.split(', ')
                id = len(self.basic_forms)
                basic_form = forms[0]
                self.basic_forms.append(basic_form)

                for form in forms:
                    self.forms[form].append(id)

    def rec(self, word):
        return self.forms[word]

    def bform(self, id):
        return self.basic_forms[id]

In [10]:
dict = Dictionary()

In [11]:
def basic_form(word):
    ids = dict.rec(word)
    return dict.bform(ids[0]) if len(ids) > 0 else word

In [12]:
print basic_form(u'drzewem')

drzewo


In [13]:
def get_basic_form(words):
    return [basic_form(word) for word in words]

In [14]:
text = [u'Jesień', u'Japonii', u'jest', u'przepiękna.']
for word in get_basic_form(text):
    print word

Jesień
Japonia
byæ
przepiękna.


In [15]:
CORPUS_PAP = 'pap/pap50000.txt'

In [ ]:
# CORPUS_PAP = 'pap/pap.txt'

In [16]:
with codecs.open(CORPUS_PAP, encoding='utf-8') as f:
    corpus = f.read().replace('\n', '')

In [ ]:
# corpus

Getting the corpus with only relevant words.

In [17]:
start = time.time()
words = preprocess_corpus(corpus)
# words
words = get_basic_form(words)
stop = time.time()
print "Preprocess time: " + str(stop-start) + " s"

Preprocess time: 5.66876316071 s


In [18]:
denominator = len(words) #number of words in corpus
denominator

232907

Count words frequency in corpus

In [19]:
word_frequency = Counter(words)
# word_frequency

In [20]:
word_frequency['rocznicy']

35

In [21]:
word_frequency['chleb']

7

In [22]:
word_frequency['powszedni']

2

In [23]:
word_frequency['masło']

0

In [24]:
word_frequency['jedzenie']

4

Count the frequency of pair words

In [25]:
def get_pairs_parallel(word):
    offset = 12
    d =[]
    index = words.index(word) #get the index of the processed word
    if(index < len(words) - offset):
        for i in range(index, index+offset): #check the neighbours of the processed word in offset distance
            pair = (word, words[i+1]) #create a pair of co-occurance
#             if(d[(words[i+1], word)] > 0): #check if there is already reversed pair
#                 pair = (words[i+1], word)
            d.append(pair)
#                 print i, pair
    return d

In [26]:
from multiprocessing import Pool
start = time.time()
p = Pool(8)
d = p.map(get_pairs_parallel, words)
stop = time.time()
print "Getting pairs parallel time: " + str(stop-start) + " s"
print d[1:5]

Getting pairs parallel time: 602.744611025 s
[[(u'\u015brody', u'czwartek'), (u'\u015brody', u'zmar\u0142'), (u'\u015brody', u'd\u0142ugiej'), (u'\u015brody', u'ci\u0119\u017ckiej'), (u'\u015brody', u'choroba'), (u'\u015brody', u'minister'), (u'\u015brody', u'kultura'), (u'\u015brody', u'dziedzictwo'), (u'\u015brody', u'narodowy'), (u'\u015brody', u'andrzej'), (u'\u015brody', u'zakrzewski'), (u'\u015brody', u'mia\u0142')], [(u'czwartek', u'zmar\u0142'), (u'czwartek', u'd\u0142ugiej'), (u'czwartek', u'ci\u0119\u017ckiej'), (u'czwartek', u'choroba'), (u'czwartek', u'minister'), (u'czwartek', u'kultura'), (u'czwartek', u'dziedzictwo'), (u'czwartek', u'narodowy'), (u'czwartek', u'andrzej'), (u'czwartek', u'zakrzewski'), (u'czwartek', u'mia\u0142'), (u'czwartek', u'wykszta\u0142cenia')], [(u'zmar\u0142', u'd\u0142ugiej'), (u'zmar\u0142', u'ci\u0119\u017ckiej'), (u'zmar\u0142', u'choroba'), (u'zmar\u0142', u'minister'), (u'zmar\u0142', u'kultura'), (u'zmar\u0142', u'dziedzictwo'), (u'zmar\u0

In [27]:
def count_pairs(lista):
    d = Counter()
    for pairs in lista:
        for pair in pairs:
            if(d[(pair[1], pair[0])] > 0): #check if there is already reversed pair
                pair = (pair[1], pair[0])
            d[pair] += 1
    return d       

In [28]:
start = time.time()
pairs_frequency = count_pairs(d)
stop = time.time()
print "Counting pairs time: " + str(stop-start) + " s"

Counting pairs time: 21.6003260612 s


In [ ]:
# pairs_frequency

In [29]:
pairs_frequency[('pomnik', 'premier')]

29

Sort the pairs according to their frequency in corpus

In [30]:
pairs_frequency.most_common(20)

[((u'czerwiec', u'polski'), 1853),
 ((u'polski', u'obch\xf3d'), 1784),
 ((u'polski', u'rocznicy'), 1784),
 ((u'polski', u'miednoje'), 1784),
 ((u'polski', u'wst\u0119pnego'), 1784),
 ((u'polski', u'\u017co\u0142nierzy'), 1784),
 ((u'polski', u'lipiec'), 1784),
 ((u'polski', u'zbrodnia'), 1784),
 ((u'polski', u'harmonogram'), 1784),
 ((u'polski', u'charkowie'), 1784),
 ((u'polski', u'katyniu'), 1784),
 ((u'polski', u'wynika\xe6'), 1784),
 ((u'proca', u'rz\u0105d'), 1294),
 ((u'proca', u'prognoza'), 1294),
 ((u'proca', u'pkb'), 1294),
 ((u'proca', u'oba'), 1294),
 ((u'proca', u'polski'), 1294),
 ((u'proca', u'komisj\u0119'), 1294),
 ((u'proca', u'strony'), 1294),
 ((u'proca', u'r'), 1294)]

Find associations's frequency for given word

In [31]:
def find_associations(pairs, word):
    counter = Counter()
    for pair in pairs.most_common():
        if (word == pair[0][0]):
            counter[pair[0][1]] = pair[1]
        if (word == pair[0][1]):
            counter[pair[0][0]] = pair[1]
    return counter

In [47]:
find_associations(pairs_frequency, "premier").most_common(20)

[(u'europejski', 443),
 (u'rocznicy', 404),
 (u'srebrny', 404),
 (u'za\u015blubin', 404),
 (u'pier\u015bcienia', 404),
 (u'pucku', 404),
 (u'zako\u0144czy\u0142y', 404),
 (u'jerzego', 404),
 (u'morze', 404),
 (u'ba\u0142tyku', 404),
 (u'obchody', 404),
 (u'buzka', 404),
 (u'polski', 404),
 (u'spotka\xe6', 303),
 (u'w\u015br\xf3d', 148),
 (u'nadal', 120),
 (u'wojny', 106),
 (u'\u015bwiatowej', 76),
 (u'wysoko\u015bci', 67),
 (u'nagrod\u0119', 60)]

Find the associantion's strenght for given word

In [33]:
def find_strenght(pairs, word, alpha = 0.66, beta = 0.00002, gamma = 0.00002):
    strenght_counter = Counter()
    for pair in pairs.most_common():
        if (word == pair[0][0]):
            association = pair[0][1]
        elif(word == pair[0][1]):
            association = pair[0][0]
        else:
            continue
        
        assoc_freq = word_frequency[association]
        pair_freq = pair[1]
        if (assoc_freq > beta * denominator):
            strenght = pair_freq / pow(assoc_freq, alpha)
        else:
            strenght = pair_freq / (gamma * denominator)
        
        strenght_counter[association] = strenght;
        
    return strenght_counter  

In [34]:
find_strenght(pairs_frequency, 'premier').most_common(10)

[(u'ba\u0142tyku', 102.40990745879294),
 (u'za\u015blubin', 86.72989648228692),
 (u'pier\u015bcienia', 86.72989648228692),
 (u'pucku', 86.72989648228692),
 (u'srebrny', 51.00831687697073),
 (u'zako\u0144czy\u0142y', 51.00831687697073),
 (u'obchody', 43.77223140627446),
 (u'morze', 39.409895189542745),
 (u'rocznicy', 38.663080690635454),
 (u'buzka', 34.82924174377278)]

In [35]:
for word in find_strenght(pairs_frequency, 'premier').most_common(10):
     print word[0],word[1]

bałtyku 102.409907459
zaślubin 86.7298964823
pierścienia 86.7298964823
pucku 86.7298964823
srebrny 51.008316877
zakończyły 51.008316877
obchody 43.7722314063
morze 39.4098951895
rocznicy 38.6630806906
buzka 34.8292417438


In [36]:
for word in find_strenght(pairs_frequency, 'ekonomiczny').most_common(10):
     print word[0],word[1]

proca 11.4303006554
priorytet 10.7200319574
podpisały 10.0664631444
uzgodniæ 8.20305926166
komisję 7.55265752228
gospodarczy 7.18110163441
wzrost 5.86711055681
tempo 5.40523092381
nt 5.30900826462
komitet 5.21915868518


In [37]:
for word in find_strenght(pairs_frequency, 'polski').most_common(10):
     print word[0],word[1]

harmonogram 493.889898474
katyniu 390.296673713
charkowie 382.985483476
wstępnego 382.985483476
miednoje 382.985483476
obchód 239.182893546
lipiec 177.490719417
rocznicy 170.730039485
czerwiec 113.301333007
żołnierzy 100.603342979


In [38]:
for word in find_strenght(pairs_frequency, 'kurs').most_common(10):
     print word[0],word[1]

zmienił 20.9866194504
spadł 16.3754551987
wzrósł 16.086577909
nfi 13.5700523627
sesja 11.2551372026
rynku 10.3343152084
papier 9.79536176424
wartościowych 9.3392972474
giełdy 9.23575596021
funduszy 8.09680097915


In [39]:
for word in find_strenght(pairs_frequency, 'minister').most_common(10):
     print word[0],word[1]

wykształcenia 144.15040009
badacz 89.520709983
zakrzewski 89.520709983
prawnik 80.8865724324
dziedzictwo 69.8097084105
historyk 48.55720108
miał 20.5039267264
historia 20.3641698354
kultura 19.0872236247
andrzej 13.9536324786


In [40]:
for word in find_strenght(pairs_frequency, 'rosja').most_common(10):
     print word[0],word[1]

zakończenie 12.4417802977
zgadzaæ 12.3228229719
wizyty 11.6240483366
robertson 10.9257331
metoda 10.9257331
stosowaæ 8.68188285116
stolicy 4.4688035216
kwietniu 3.78137546036
konflikt 3.59235522648
kampanię 3.10480072152


In [41]:
for word in find_strenght(pairs_frequency, 'niemcy').most_common(10):
    print word[0],word[1]

przyjaźni 15.9376218581
służyć 12.1956032512
cudowny 11.1632540027
mogącej 11.1632540027
przyjaciół 10.6827772145
przykład 8.70528737972
najważniejszych 6.76089785056
izraela 5.50938745164
strefy 3.10480072152
mówił 2.93238443662


In [42]:
for word in find_strenght(pairs_frequency, 'polityka').most_common(10):
    print word[0],word[1]

wyrównywaæ 4.72291515498
groźby 4.72291515498
mogącej 4.72291515498
wspaniałą 4.72291515498
spowodować 4.26739710674
wybuch 3.39098909164
szansę 2.94956483072
braku 2.76917370155
uwolniæ 2.38363636371
świat 2.33089469108


In [48]:
for word in find_strenght(pairs_frequency, u'mas\u0142o').most_common(10):
    print word[0],word[1]

In [44]:
for word in find_strenght(pairs_frequency, 'jedzenie').most_common(10):
    print word[0],word[1]

dawka 1.22597091217
trzydziestu 0.858711846359
witaminą 0.858711846359
spożyciu 0.858711846359
tabletka 0.858711846359
wywoływało 0.858711846359
witaminę 0.858711846359
zwiększonej 0.858711846359
pożywienia 0.858711846359
bogaty 0.82175209342


In [45]:
for word in find_strenght(pairs_frequency, 'chleb').most_common(10):
    print word[0],word[1]

procesy 2.4197908888
pustynia 2.4197908888
ofiarę 2.14544909629
egipcie 1.93790879446
zasypującej 1.50274573113
oazę 1.50274573113
charga 1.50274573113
badała 1.50274573113
zachodzące 1.50274573113
bóstwo 1.50274573113


In [46]:
for word in find_strenght(pairs_frequency, 'powszedni').most_common(10):
    print word[0],word[1]

poważniejszych 0.644033884769
uchwaliła 0.612985456083
taksówką 0.42935592318
taksówkowy 0.42935592318
zobowiązała 0.42935592318
usługi 0.334818745374
maksymalny 0.320856474391
ortopedyczny 0.21467796159
ortopeda 0.21467796159
warszawy 0.127205878112
